In [1]:
import tensorflow as tf #Red neuronal convolucional
import pandas as pd #Visualizar data
import numpy as np #La libreria MNIST con los datos esta en formato numpy
import matplotlib.pyplot as plt #Graficas
from tensorflow.keras.models import Sequential #Tipo de modelo secuelcial, para agruegar capas
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
#Dense para crear capas full conected (RNFC)
#Dropout para evitar el overfiting, desactivando un porcetaje de neuronas determinado
#Flatten para convetir las salidas en un vector 1D
#Conv2D para relizar operaciones de convolucion
#MaxPooling para realizar operaciones de Maxpooling

In [2]:
mnist_data = tf.keras.datasets.mnist

#data de entrenamiento y data de testeo
(train_images, train_labels),(test_images, test_labels) = mnist_data.load_data()

#este set de datos ya viene con data de entrenamiento y de test separados
print("Dimensiones del set de entrenamiento:", train_images.shape)
print("Imagenes de entrenamiento:", train_images.shape[0])
print("Imagenes de testeo:", test_images.shape[0])

Dimensiones del set de entrenamiento: (60000, 28, 28)
Imagenes de entrenamiento: 60000
Imagenes de testeo: 10000


In [3]:
#Variables

#cantidad de digitos a clasificar
num_classes = 10
#tamanio de cada subconjunto (para no agarra las 60000 y llenar la memoria)
batch_size = 128
#cuanta veces va a recorrer todo el cojunto de entrenamiento
epochs = 5
#forma de las imagenes
input_shape = (28, 28, 1)

In [4]:
#Nomalizamos los valores entre 0 y 1
train_images = train_images.astype('float32')
test_images = test_images.astype('float32')
train_images /= 255
test_images /= 255
print("ValMinTR", np.amin(train_images))
print("ValMinTE", np.amin(test_images))
print("ValMaxTR", np.amax(train_images))
print("ValMaxTE", np.amax(test_images))
print("")

#Establecemos el numero de canales en 1 ya que la imgen esta en escala de grises
train_images = train_images.reshape(60000, 28, 28, 1)
test_images = test_images.reshape(10000, 28, 28, 1)
print("Dim", train_images.shape)
print("Dim", test_images.shape)
print("")

#Convertimos los vectores de clase en matrices binarias
print(test_labels)
train_labels = tf.keras.utils.to_categorical(train_labels, num_classes)
test_labels = tf.keras.utils.to_categorical(test_labels, num_classes)
print(test_labels)

ValMinTR 0.0
ValMinTE 0.0
ValMaxTR 1.0
ValMaxTE 1.0

Dim (60000, 28, 28, 1)
Dim (10000, 28, 28, 1)

[7 2 1 ... 4 5 6]
[[0. 0. 0. ... 1. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [5]:
#Creacion del modelo
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), #32 filtros de 3x3
                 activation="relu", 
                 input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, kernel_size=(3, 3), activation="relu")) #64 filtros de 3x3
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten()) #Flatten para covertir el vector en uno unidemensional
model.add(Dropout(0.5)) #Apagamo el 50% de las neuronas para reforzar el aprendisaje
model.add(Dense(num_classes, activation="softmax")) 
#esto hara que cada salida tenga un valor entre 0 y 1 de tipo probabilistico

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 1600)              0         
                                                                 
 dropout (Dropout)           (None, 1600)              0

In [6]:
#Compilamos el modelo
model.compile(loss="categorical_crossentropy", 
              optimizer="adam", 
              metrics=["accuracy"])

#Entrenamiento
entrenado = model.fit(train_images, train_labels, 
          batch_size=batch_size, 
          epochs=epochs, 
          verbose=1)

Epoch 1/5
469/469 [==============================] - 9s 18ms/step - loss: 0.3542 - accuracy: 0.8920
Epoch 2/5
469/469 [==============================] - 8s 18ms/step - loss: 0.1016 - accuracy: 0.9687
Epoch 3/5
469/469 [==============================] - 8s 18ms/step - loss: 0.0753 - accuracy: 0.9773
Epoch 4/5
469/469 [==============================] - 8s 18ms/step - loss: 0.0655 - accuracy: 0.9797
Epoch 5/5
469/469 [==============================] - 9s 19ms/step - loss: 0.0572 - accuracy: 0.9820


In [7]:
#Validation data
test_loss, test_accuracy = model.evaluate(test_images, test_labels, verbose=1)
print(f"TEST LOSS: {test_loss}")
print(f"TEST ACCURACY: {test_accuracy}")
print("")  

#Historial
frame = pd.DataFrame(entrenado.history)
print("PRECISION POR EPOCAS")
print(frame)

313/313 [==============================] - 1s 2ms/step - loss: 0.0325 - accuracy: 0.9891
TEST LOSS: 0.03251289203763008
TEST ACCURACY: 0.9890999794006348

PRECISION POR EPOCAS
       loss  accuracy
0  0.354207  0.892017
1  0.101618  0.968717
2  0.075293  0.977317
3  0.065530  0.979667
4  0.057199  0.981983
